Professor Edson Cilos

Linkedin: https://www.linkedin.com/in/edson-cilos-032a66162/

Website: https://edsoncilos.com

Esse material é explicado com detalhes no meu [curso](https://www.udemy.com/course/edson-cilos-ml/?referralCode=2C9C581FAB301BBAE173).

**Análise de microplásticos no oceano**

---

Nesta atividade vamos treinar um modelo para caraterização de microplásticos no oceanos

In [ ]:
# Importar algumas das bibliotecas importantes

#sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

#Computação científica
from scipy.sparse.linalg import spsolve
from scipy import sparse

# pandas e numpy
import pandas as pd
import numpy as np

# Download arquivos
from six.moves import urllib

seed = 10

In [ ]:
link = "https://raw.githubusercontent.com/EdsonCilos/mp_classification/\
master/data/d4_rebuild.csv"

save_path = "data.csv"

def save_file_from_url(link = link, save_path=save_path):
    urllib.request.urlretrieve(link, save_path)

In [ ]:
save_file_from_url()

In [ ]:
data = pd.read_csv(save_path)

# Conhecendo os dados

In [ ]:
data

Na nossa aplicação a coluna "Nom" não tem aplicação para fins preditivos

In [ ]:
data.drop(['Nom '], axis=1, inplace=True)

Vamos agora renomear os nosso rótulos (opcional)

In [ ]:
data.rename(columns={'Interpretation ': "label"}, inplace=True)

Vamos estudar a distribuição dos rótulos

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
data["label"].hist(bins=50, figsize=(30,15))
plt.show()

In [ ]:
data["label"].value_counts()

# Preparando os dados

Vamos verificar a presença de dados faltantes

In [ ]:
len(data[data.isnull().any(axis=1)])

Vamos incluir todas as classes pouco representativas em Unknown

In [ ]:
def transform_less_representative(data, threshold = 5):

  dataset = data.copy()

  freq = dataset["label"].value_counts()
  less_rep = [idx for idx, value in freq.items()  if value < threshold]
  
  for i, row in dataset.iterrows():
      if row["label"] in less_rep:
          dataset.at[i, 'label'] = 'Unknown'

  return dataset



In [ ]:
df = transform_less_representative(data)

In [ ]:
df["label"].value_counts()

Vamos conferir rapidinho os tipos de dados

In [ ]:
df.dtypes

In [ ]:
df.dtypes.unique()

Vamos agora codificar os rótulos do problema

In [ ]:
encoder = LabelEncoder()
df["label"] = encoder.fit_transform(df["label"])

Vamos aproveitar e revisar um pouco sobre os codificadores

In [ ]:
df["label"].unique()

In [ ]:
encoder.inverse_transform(range(14))

In [ ]:
encoder.inverse_transform([0])

In [ ]:
encoder.inverse_transform([13])

In [ ]:
encoder.inverse_transform([1, 1, 7, 3, 3, 8])

Vamos agora criar a matriz de características e o vetor de rótulos

In [ ]:
X = df.drop(["label"], axis = 1)
y = df["label"].copy()

Separando em treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=1/3,
                                                    random_state = seed)

# Treinando primeiro modelo

Vamos começar treinando um SVM com kernel linear. Faremos isso usando de diferentes formas:



*   Sem escalonamento de atributos
*   Com escalonamento de atributos, de maneira rigorosa
*   Com escalonamento de atributos, de forma enviesada


In [ ]:
#Classificador base
clf = SVC(kernel='linear', C=0.0015, probability=True)

scoring = {'acc': 'accuracy', 
           'neg_loss': 'neg_log_loss'}

In [ ]:
#Sem escalonamento
score = cross_validate(clf, X_train, y_train, 
                        scoring=scoring, cv=5)

print("Log loss: " + str(np.round(-np.mean(score['test_neg_loss']), 3)))
print("Acurácia: " + str(np.round(100*np.mean(score['test_acc']), 2)) + "%")

In [ ]:
# Pipeline rigoroso
pipe = Pipeline([
                 ('std_scaler', StandardScaler()),
                 ('classifier', clf)
                 ])

score = cross_validate(pipe, X_train, y_train, 
                        scoring=scoring, cv=5)

print("Log loss: " + str(np.round(-np.mean(score['test_neg_loss']), 3)))
print("Acurácia: " + str(np.round(100*np.mean(score['test_acc']), 2)) + "%")

In [ ]:
# Metodologia enviesada
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X_train)

score = cross_validate(clf, X_scaled, y_train, 
                        scoring=scoring, cv=5)

print("Log loss: " + str(np.round(-np.mean(score['test_neg_loss']), 3)))
print("Acurácia: " + str(np.round(100*np.mean(score['test_acc']), 2)) + "%")


# Consulte os especialistas!

É comum fazer a "correção da linha de base" dentro de técnicas de FTIR (espectroscopia de infravermelho por transformada de Fourier)

In [ ]:
def als(y, lam=1e5, p=0.01, itermax=10):
    """
    Implements an Asymmetric Least Squares Smoothing
    baseline correction algorithm (P. Eilers, H. Boelens 2005)

    Baseline Correction with Asymmetric Least Squares Smoothing
    based on https://github.com/vicngtor/BaySpecPlots

    Baseline Correction with Asymmetric Least Squares Smoothing
    Paul H. C. Eilers and Hans F.M. Boelens
    October 21, 2005

    Description from the original documentation:

    Most baseline problems in instrumental methods are characterized by a
    smooth baseline and a superimposed signal that carries the analytical 
    information: a series of peaks that are either all positive or all 
    negative. We combine a smoother with asymmetric weighting of deviations 
    from the (smooth) trend get an effective baseline estimator. 
    It is easy to use, fast and keeps the analytical peak signal intact.
    No prior information about peak shapes or baseline (polynomial) is needed
    by the method. The performance is illustrated by simulation and 
    applications to real data.

    Inputs:
        y:
            input data (i.e. chromatogram of spectrum)
        lam:
            parameter that can be adjusted by user. The larger lambda is,
            the smoother the resulting background, z
        p:
            wheighting deviations. 0.5 = symmetric, <0.5: negative
            deviations are stronger suppressed
        itermax:
            number of iterations to perform
    Output:
        the fitted background vector

    """
    L = len(y)
    D = sparse.eye(L, format='csc')
    D = D[1:] - D[:-1]  
    D = D[1:] - D[:-1]
    D = D.T
    w = np.ones(L)
    for i in range(itermax):
        W = sparse.diags(w, 0, shape=(L, L))
        Z = W + lam * D.dot(D.T)
        z = spsolve(Z, w * y)
        w = p * (y > z) + (1 - p) * (y < z)
    return z

In [ ]:
def plot_sample(sample, sample2 = None, baseline=True):

    horizontal = [int(x.split('.')[0]) for x in sample.index.values]

    values = sample.values

    plt.xlabel("Wavelenght (1/cm)")

    plt.plot(horizontal, values, color = (1/255,209/255,209/255), 
             label = 'sample')
    
    if sample2 is not None:
      plt.plot(horizontal, sample2.values, color = 'red', label = 'sample 2')
    
    if baseline:
      plt.plot(horizontal, values - als(values), 
              color = (90/255, 53/255, 182/255), 
              label= 'sample (corrected)')
             
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()
    plt.close()

In [ ]:
plot_sample(X_train.iloc[0])

Vamos agora aplicar a nossa correção de base para todos os dados:

**OBS:** A correção de cada instância não usa informação do restante dos dados, de forma que a correção pode ser feita em todos os dados sem introduzir viés / data leakage

In [ ]:
print("Aplicando correção de linha de base...")

old_sample = X_train.iloc[0].copy()


print("Transformando conjunto de treino")
for idx, row in X_train.iterrows():
  X_train.loc[idx, :] = row - als(row)


print("Transformando conjunto de teste")
for idx, row in X_test.iterrows():
  X_test.loc[idx, :] = row - als(row)

print("Processo finalizado!")

In [ ]:
plot_sample(old_sample, baseline=False)

In [ ]:
plot_sample(X_train.iloc[0], baseline=False)

Vamos repetir a etapa de escalonamento:

In [ ]:
#Sem escalonamento
score = cross_validate(clf, X_train, y_train, 
                        scoring=scoring, cv=5)

print("Log loss: " + str(np.round(-np.mean(score['test_neg_loss']), 3)))
print("Acurácia: " + str(np.round(100*np.mean(score['test_acc']), 2)) + "%")

In [ ]:
# Pipeline rigoroso
pipe = Pipeline([
                 ('std_scaler', StandardScaler()),
                 ('classifier', clf)
                 ])

score = cross_validate(pipe, X_train, y_train, 
                        scoring=scoring, cv=5)

print("Log loss: " + str(np.round(-np.mean(score['test_neg_loss']), 3)))
print("Acurácia: " + str(np.round(100*np.mean(score['test_acc']), 2)) + "%")

Ganhamos 7% só ao usar a metodologia correta para preparar os dados!

In [ ]:
# Metodologia enviesada
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X_train)

score = cross_validate(clf, X_scaled, y_train, 
                        scoring=scoring, cv=5)

print("Log loss: " + str(np.round(-np.mean(score['test_neg_loss']), 3)))
print("Acurácia: " + str(np.round(100*np.mean(score['test_acc']), 2)) + "%")

A metologia errada continua aumentando a performance do modelo, ainda que marginal.

# Análise de componentes principais

Vamos aplicar a análise de componentes principais para redução de dimensionalidade

In [ ]:
# Pipeline rigoroso
pipe = Pipeline([
                 ('std_scaler', StandardScaler()),
                 ("dim_red", PCA(n_components=100, random_state=seed)),
                 ('classifier', clf)
                 ])

score = cross_validate(pipe, X_train, y_train, 
                        scoring=scoring, cv=5)

print("Log loss: " + str(np.round(-np.mean(score['test_neg_loss']), 3)))
print("Acurácia: " + str(np.round(100*np.mean(score['test_acc']), 2)) + "%")

In [ ]:
# Metodologia enviesada
scaler = StandardScaler()
n_components= 100
pca = PCA(n_components= n_components, random_state=seed)

X_transformed = pca.fit_transform(scaler.fit_transform(X_train))

score = cross_validate(clf, X_transformed, y_train, 
                        scoring=scoring, cv=5)

print("Log loss: " + str(np.round(-np.mean(score['test_neg_loss']), 3)))
print("Acurácia: " + str(np.round(100*np.mean(score['test_acc']), 2)) + "%")

Vamos por enquanto esquecer a validação cruzada e explorar o conjunto de dados transformado via escalonamento + std scalar

In [ ]:
pca.explained_variance_ratio_

Organizando a informação

In [ ]:
explained_ratio = np.round(np.sum(pca.explained_variance_ratio_)*100, 3)

print("Com {} componentes: {} % da variância explicada".format(
    n_components, explained_ratio))


Vamos ver a contribuição acumulada:

In [ ]:
np.round(np.cumsum(pca.explained_variance_ratio_)*100, 3)

Vamos determinar o número mínimo de componentes para explicar 99% das característica:

In [ ]:
pca = PCA(n_components= n_components, random_state=seed)
scaler = StandardScaler()
X_transformed = pca.fit_transform(scaler.fit_transform(X_train))
cum_sum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cum_sum >= 0.99) + 1
print(d)

Porém existe uma maneira bem mais direta de fazer isto:

In [ ]:
pca = PCA(n_components= 0.99, random_state=seed)
scaler = StandardScaler()
X_transformed = pca.fit_transform(scaler.fit_transform(X_train))
print(np.round(np.sum(pca.explained_variance_ratio_)*100, 3))

In [ ]:
pca.n_components_

In [ ]:
def plot_dimension_vs_explained_variance(X_data):

    pca = PCA(n_components= 100, random_state=seed)
    scaler = StandardScaler()
    X_transformed = pca.fit_transform(scaler.fit_transform(X_data))

    cum_sum = np.cumsum(pca.explained_variance_ratio_)

    plt.xlabel("Dimensões")
    plt.ylabel("Variância explicada")

    plt.plot(range(len(cum_sum)), 
             cum_sum, 
             color = (1/255,209/255,209/255))

    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()
    plt.close()

In [ ]:
plot_dimension_vs_explained_variance(X_train)

In [ ]:
pca = PCA(n_components= 25, random_state=seed)
scaler = StandardScaler()
X_transformed = pca.fit_transform(scaler.fit_transform(X_train))
print(np.round(np.sum(pca.explained_variance_ratio_)*100, 3))

# PCA para compressão

Vamos revisar a nossa amostra de exemplo:

In [ ]:
plot_sample(X_train.iloc[0], baseline=False)

In [ ]:
pca = PCA(n_components= 46, random_state=seed)
scaler = StandardScaler()
X_transformed = pca.fit_transform(scaler.fit_transform(X_train))

In [ ]:
X_recovered = pd.DataFrame(
    data = scaler.inverse_transform(pca.inverse_transform(X_transformed)),
    columns = X_train.columns
)

In [ ]:
plot_sample(sample = X_train.iloc[0],  
            sample2 = X_recovered.iloc[0],
            baseline=False)

In [ ]:
X_train.info()

In [ ]:
pd.DataFrame(data = X_transformed).info()

Será que vale a pena essa compressão?

In [ ]:
# Pipeline rigoroso
pipe = Pipeline([
                 ('std_scaler', StandardScaler()),
                 ("dim_red", PCA(n_components=46, random_state=seed)),
                 ('classifier', clf)
                 ])

score = cross_validate(pipe, X_train, y_train, 
                        scoring=scoring, cv=5)

print("Log loss: " + str(np.round(-np.mean(score['test_neg_loss']), 3)))
print("Acurácia: " + str(np.round(100*np.mean(score['test_acc']), 2)) + "%")

# Avaliação final

In [ ]:
model = Pipeline([
                 ('std_scaler', StandardScaler()),
                 ("dim_red", PCA(n_components=46, random_state=seed)),
                 ('classifier', clf)
                 ])

model.fit(X_train, y_train)



In [ ]:
log_loss(y_test, model.predict_proba(X_test))

In [ ]:
accuracy_score(np.array(y_test), model.predict(X_test))

In [ ]:
import pickle

pickle.dump(encoder, open('encoder.sav', 'wb'))
pickle.dump(model, open('classifier.sav', 'wb'))

# Bonus

In [ ]:
from PIL import Image
import requests
from io import BytesIO

link_sample = "https://raw.githubusercontent.com/EdsonCilos/mlcourse/master/\
reducao_dimensionalidade/PCA/prediction_132.png"

response = requests.get(link_sample)
img = Image.open(BytesIO(response.content))

In [ ]:
img